In [6]:
#imports
from __future__ import division, print_function
import sympy as sm
from sympy import trigsimp
import sympy.physics.mechanics as me
import mpmath as mp
import numpy as np

from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)

# Defining symbolic Variables

n = 1
q = me.dynamicsymbols('q:{}'.format(n + 1))  # generalized coordinates
qdot = me.dynamicsymbols('qdot:{}'.format(n + 1))  #generalized speeds
xx = q + qdot
f = me.dynamicsymbols('f')
m = sm.symbols('m:{}'.format(n + 1))
J = sm.symbols('J:{}'.format(n + 1))
l = sm.symbols('l:{}'.format(n))  # lenght of each pendlum
a = sm.symbols('a:{}'.format(n))  #location of Mass-centers
d = sm.symbols('d1:{}'.format(n + 1))  #viscous damping coef.
g, t = sm.symbols('g t')

# intertial reference frame
In_frame = me.ReferenceFrame('In_frame')

# Origninal Point O in Reference Frame :
O = me.Point('O')
O.set_vel(In_frame, 0)

# The Center of Mass Point on cart :
C0 = me.Point('C0')
C0.set_pos(O, q[0] * In_frame.x)
C0.set_vel(In_frame, qdot[0] * In_frame.x)

cart_inertia_dyadic = me.inertia(In_frame, 0, 0, J[0])
cart_central_intertia = (cart_inertia_dyadic, C0)

cart = me.RigidBody('Cart', C0, In_frame, m[0], cart_central_intertia)

kindiffs = [q[0].diff(t) - qdot[0]]  # entforcing qdot=Omega

frames = [In_frame]
mass_centers = [C0]
joint_centers = [C0]
central_intertias = [cart_central_intertia]

forces = [(C0, f * In_frame.x - m[0] * g * In_frame.y)]

torques=[]

rigid_bodies = [cart]


for i in range(n):
    #Creating new reference frame
    Li = In_frame.orientnew('L' + str(i), 'Axis', [q[i + 1], In_frame.z])
    Li.set_ang_vel(In_frame, qdot[i + 1] * In_frame.z)
    frames.append(Li)

    # Creating new mass point centers
    Pi = mass_centers[-1].locatenew('a' + str(i + 1), a[i] * Li.x)
    Pi.v2pt_theory(joint_centers[-1], In_frame, Li)
    mass_centers.append(Pi)

    #Creating new joint Points
    Jointi = joint_centers[-1].locatenew('jont' + str(i + 1), l[i] * Li.x)
    Jointi.v2pt_theory(joint_centers[-1], In_frame, Li)
    joint_centers.append(Jointi)

    #adding forces
    forces.append((Pi, -m[i + 1] * g * In_frame.y))
    
    #adding torqes
    if i==0 :
        torqueVectori= -d[0] * qdot[1] * In_frame.z
        torques.append((Li, torqueVectori))
    else:
        torqueVectori = -d[i] * (qdot[i+1]-qdot[i])*In_frame.z
        torques.append((Li, torqueVectori))
    
    
    #adding cential inertias 
    IDi = me.inertia(frames[i + 1], 0, 0, J[i + 1])
    ICi = (IDi, mass_centers[i + 1])
    central_intertias.append(ICi)

    LBodyi = me.RigidBody('L' + str(i + 1) + '_Body', mass_centers[i + 1],
                          frames[i + 1], m[i + 1], central_intertias[i + 1])
    rigid_bodies.append(LBodyi)

    kindiffs.append(q[i + 1].diff(t) - qdot[i + 1])
    

#generalized force
loads = torques + forces

#Kane's Method --> Equation of motion
Kane = me.KanesMethod(In_frame, q, qdot, kindiffs)
fr, frstar = Kane.kanes_equations(rigid_bodies, loads)

mass_matrix = trigsimp(Kane.mass_matrix_full)
forcing_vector = trigsimp(Kane.forcing_full)

    

In [7]:
xx

[q0, q1, qdot0, qdot1]

In [8]:
constants = [l, a[0], m[0], m[1], J[0], J[1], d[0], g]
specified=[f]

In [9]:
numerical_constants = np.array([
    0.32,  # l0
    0.2,  #a0
    3.34,  #m0
    0.8512,  #m1
    0,  #J0
    0.01980,  #J1
    0.00715,  #d
    9.81  #g
])

#
numerical_specified = np.array([0])

In [10]:
equilibrium_dict = {q[0]: 0, q[1]: np.pi / 2, qdot[0]:0, qdot[1]:0}
parameter_dict = dict(zip(constants, numerical_constants))
op_point=[equilibrium_dict, parameter_dict]

In [11]:
mass_matrix

Matrix([
[1, 0,              0,              0],
[0, 1,              0,              0],
[0, 0,        m0 + m1, -a0*m1*sin(q1)],
[0, 0, -a0*m1*sin(q1),  J1 + a0**2*m1]])

In [12]:
x2= sm.Matrix([q[0], q[1]])
mass_matrix.jacobian(x2)

TypeError: self must be a row or a column matrix

In [13]:
forcing_vector

Matrix([
[                        qdot0],
[                        qdot1],
[   a0*m1*qdot1**2*cos(q1) + f],
[-(a0*g*m1*cos(q1) + d1*qdot1)]])

In [14]:
mass_matrix.inv()*forcing_vector

Matrix([
[                                                                                                                                                                                                 qdot0],
[                                                                                                                                                                                                 qdot1],
[-a0*m1*(a0*g*m1*cos(q1) + d1*qdot1)*sin(q1)/(-a0**2*m1**2*sin(q1)**2 + (J1 + a0**2*m1)*(m0 + m1)) + (J1 + a0**2*m1)*(a0*m1*qdot1**2*cos(q1) + f)/(-a0**2*m1**2*sin(q1)**2 + (J1 + a0**2*m1)*(m0 + m1))],
[       a0*m1*(a0*m1*qdot1**2*cos(q1) + f)*sin(q1)/(-a0**2*m1**2*sin(q1)**2 + (J1 + a0**2*m1)*(m0 + m1)) - (m0 + m1)*(a0*g*m1*cos(q1) + d1*qdot1)/(-a0**2*m1**2*sin(q1)**2 + (J1 + a0**2*m1)*(m0 + m1))]])

In [15]:


import sympy as sp
from sympy import sin, cos, pi
from sympy.interactive import printing
import pickle
import numpy as np
import scipy as sc
import scipy.interpolate
from scipy.integrate import odeint
import matplotlib.pyplot as pl

import symbtools as st
import symbtools.modeltools as mt
import symbtools.noncommutativetools as nct

import pycartan as pc
zip0 = st.zip0
printing.init_printing(1)

In [19]:
st.make_global(q, qdot,m)

In [30]:
T=1/2*m[1]*mass_centers[1].vel(In_frame).dot(mass_centers[1].vel(In_frame))

In [29]:
V=0

In [33]:
mod = mt.generate_symbolic_model(T, V, xx, [0, 0])

AssertionError: 